In [1]:
import pandas as pd

In [2]:
from iac_sketch import data, parse, sketch

In [9]:
# DEBUG
import importlib

importlib.reload(data)
importlib.reload(parse)
importlib.reload(sketch)
architect = sketch.Architect("../public/components")
registry = architect.parse()

In [10]:
links = registry["links"]
links

,entity,comp_ind,links,link_type
0,request_workflow,1,accept_request --> build_report\nbuild_report ...,dependency
1,build_report,2,define_report --> set_up_environment\nset_up_e...,dependency
2,run_report,2,run_etl --> run_cohort_notebook\nrun_cohort_no...,dependency


In [11]:
registry["metadata"]

,entity,comp_ind,source_file,n_comps,metadata
0,request_workflow,2,../public/components/workflow.yaml,3,NaN
1,build_report,3,../public/components/workflow.yaml,4,NaN
2,define_report,2,../public/components/workflow.yaml,3,NaN
3,set_up_environment,2,../public/components/workflow.yaml,3,NaN
4,set_up_report,3,../public/components/workflow.yaml,4,NaN
...,...,...,...,...,...
75,variable_names,1,../public/components/conventions.yaml,2,NaN
76,basic_report_test,4,../public/components/testcases.yaml,5,NaN
77,no_irb_test,4,../public/components/testcases.yaml,5,NaN
78,updated_report_test,4,../public/components/testcases.yaml,5,NaN


In [16]:
link_df = registry["link"].set_index(["entity", "comp_ind"])
link_df

link_type                  source  \
entity           comp_ind                                       
request_workflow 3         dependency          accept_request   
                 4         dependency            build_report   
build_report     4         dependency           define_report   
                 5         dependency      set_up_environment   
                 6         dependency           set_up_report   
                 7         dependency          work_on_report   
                 8         dependency  get_feedback_on_report   
                 9         dependency  get_feedback_on_report   
run_report       4         dependency                 run_etl   
                 5         dependency     run_cohort_notebook   

                                           target  
entity           comp_ind                          
request_workflow 3                   build_report  
                 4                 deliver_report  
build_report     4             set_up_environment  
                 5                  set_up_report  
                 6                 work_on_report  
                 7         get_feedback_on_report  
                 8                 work_on_report  
                 9                finalize_report  
run_report       4            run_cohort_notebook  
                 5            run_report_notebook

In [21]:
metadata_df = registry["metadata"].set_index(["entity", "comp_ind"])
metadata_df

,,source_file,n_comps,metadata
entity,comp_ind,,,
request_workflow,2,../public/components/workflow.yaml,3,NaN
build_report,3,../public/components/workflow.yaml,4,NaN
define_report,2,../public/components/workflow.yaml,3,NaN
set_up_environment,2,../public/components/workflow.yaml,3,NaN
set_up_report,3,../public/components/workflow.yaml,4,NaN
...,...,...,...,...
variable_names,1,../public/components/conventions.yaml,2,NaN
basic_report_test,4,../public/components/testcases.yaml,5,NaN
no_irb_test,4,../public/components/testcases.yaml,5,NaN


In [ ]:
n_new_comps

entity
build_report        6
request_workflow    2
run_report          2
Name: count, dtype: int64

In [25]:
link_df.join(metadata_df, on="entity", how="inner", lsuffix='_link', rsuffix='_metadata', level=0)

TypeError: DataFrame.join() got an unexpected keyword argument 'level'

In [8]:
# Find invalid requirements
reqs = registry.view(["requirement", "status", "task", "input"])
is_valid = reqs["status"].isin(["closed", "removed"])
invalid_reqs = reqs.loc[~is_valid]
invalid_reqs

,entity,status,requester [person],input
0,accept_request,in design,The person seeking to submit a request.,NaN
1,deliver_report,new,NaN,NaN
2,archive_data_products,new,NaN,NaN
3,framework_manifest_data,in design,NaN,NaN
4,framework_manifest_documentation,new,NaN,NaN
5,power_users,new,NaN,NaN
6,audit_cohorts,new,NaN,NaN


In [62]:
# Find invalid testcases
registry.view(["testcase", "status"])

,entity,status
0,basic_report_test,new
1,no_irb_test,new
2,updated_report_test,new
3,updated_report_new_analyst_test,new
